In [19]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
# Get the wikipedia page containg the list
url = 'https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')

In [7]:
# Fetch the table containing the revenue sorting
# All table on page have the same css class. The one is question is the first one --> soup.find()
table = soup.find('table', class_='wikitable sortable')

In [90]:
# Extract the table headers, drop html tags and special symbols
table_headers = table.find_all('th')
titles = [title.text.strip() for title in table_headers]
titles

['Rank',
 'Name',
 'Industry',
 'Revenue (USD millions)',
 'Revenue growth',
 'Employees',
 'Headquarters']

In [115]:
df = pd.DataFrame(columns=titles)
len(df)

0

In [123]:
# Extract the table rows, then data in them.
# Populate the dataframe and make 'Rank' the index
table_rows = table.find_all('tr')

for row in table_rows[1:]:
    row_data = row.find_all('td')
    indiv_row_data = [data.text.strip() for data in row_data]
    
    df.loc[len(df)] = indiv_row_data
    
df.set_index('Rank', inplace=True)    
df

,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters
Rank,,,,,,
1,Walmart,Retail,"611,289",6.7%,"2,100,000","Bentonville, Arkansas"
2,Amazon,Retail and Cloud Computing,"513,983",9.4%,"1,540,000","Seattle, Washington"
3,Exxon Mobil,Petroleum industry,"413,680",44.8%,"62,000","Spring, Texas"
4,Apple,Electronics industry,"394,328",7.8%,"164,000","Cupertino, California"
5,UnitedHealth Group,Healthcare,"324,162",12.7%,"400,000","Minnetonka, Minnesota"
...,...,...,...,...,...,...
96,Best Buy,Retail,"46,298",10.6%,"71,100","Richfield, Minnesota"
97,Bristol-Myers Squibb,Pharmaceutical industry,"46,159",0.5%,"34,300","New York City, New York"
98,United Airlines,Airline,"44,955",82.5%,"92,795","Chicago, Illinois"


In [126]:
# Save frame into a CSV
df.to_csv(r'companies.csv')